# Report 3

### by Kaitlyn Keil and Kevin Zhang
### April 2017

<a href = http://www.nature.com/articles/srep00196>Flavor network and the principles of food pairing</a>

In [50]:
""" This file contains code used to analyze the
flavor pairing dataset, the information for which can be found
at this link: http://www.nature.com/articles/srep00196

The following packages are required to run this code.
"""

from __future__ import print_function, division
import pandas as pd

import sys
import numpy as np
import math
import thinkstats2
import thinkplot
import matplotlib.pyplot as plt
from operator import itemgetter
from cycler import cycler
from sklearn.feature_extraction import DictVectorizer

from collections import defaultdict, Counter
import survival
import statsmodels.formula.api as smf

In [166]:
### This is for the other massive set

def ReadIngredients(region='African'):
    """ Takes the ingedients dataset and converts it into a Dataframe. 
    Region can be: African, EastAsian, EasternEuropean, LatinAmerican,
        MiddleEastern, NorthAmerican, NorthernEuropean, SouthAsian,
        SoutheastAsian, SouthernEuropean, WesternEuropean"""
    temp = pd.read_csv("FoodDataset/ingredients.csv") # Reads the file with a lot of NaNs and little order
    #temp.fillna(value='None')
    temp2 = temp[temp.cuisine_type==region]
    temp_dict = temp2.to_dict('index')
    
    temp_list = []
    
    for keys,values in temp_dict.items():
        td = {}
        for key,x in values.items():
            if key=='cuisine_type':
                td[key] = x
            else:
                td[x]=1
        temp_list.append(td)
        
        
    return pd.DataFrame(temp_list).fillna(0)

def corrPlot(df):
    """ Given the ingredients dataframe, selects
    the most used ingredients and creates a correlation 
    matrix from them."""
    df1 = df.ix[:, (df.columns != 'cuisine_type')]
    temp_df = df1.sum()
    mdf = temp_df[temp_df > 50].iloc[1:]
    print(mdf)
    corr_df = pd.DataFrame()
    
    for x in mdf.index:
        if x!='index1':
            corr_df[x] = df[x]
        
    corr_df['index1'] = [int(x) for x in corr_df.index.values]
    sns.heatmap(corr_df.corr())
    return corr_df

#ingredients = ReadIngredients('SoutheastAsian')

# Shows what ingredients are most/least likely to be used together.
#  For example, in African cuisine, cumin and coriander are fairly 
#  correlated, whereas vegetable oil and olive oil together are not.
#%matplotlib inline
#corr_df = corrPlot(ingredients)

In [179]:
tester = pd.read_excel('FoodDataset/dietary.xls', sheetname='Proximates')

# TO DO
# For cleaning: 
#  Get rid of Ns and Trs
#  Convert rows into floats
#  Get ready for lots of brackets...
#  Potentially just make different columns for the integer values
#  Try to get a cleaner way to exclude alcohol

# For report:
#  Figure out if we can actually classify based on these values
#  Make a dictionary mapping from the Group codes to actual names
#  Figure out a good model
#  Do the thing

In [180]:
tester

,Food Code,Food Name,Description,Group,Constant,Previous,Comments,Description Footnote,Water (g),Total nitrogen (g),...,cis-Poly FA /100g Food (g),Poly FA /100g FA (g),Poly FA /100g food (g),Sat FA excl Br /100g FA (g),Sat FA excl Br /100g food (g),Branched chain FA /100g FA (g),Branched chain FA /100g food (g),Trans FAs /100g FA (g),Trans FAs /100g food (g),Cholesterol (mg)
0,17-215,"Beer, mild, draught",Mixed sample from different brewers,QA,894.0,50-1097,"As MSF 17-215 - Mild, draught. Additional data...",NaN,(95.0),0.03,...,NaN,NaN,Tr,NaN,NaN,NaN,NaN,NaN,0.00,0.0
1,50-1094,"Beer, bitter, canned",6 samples,QA,892.0,NaN,NaN,NaN,(95.8),0.04,...,NaN,NaN,Tr,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,17-506,"Beer, bitter, average","5 samples from different brewers; canned, drau...",QA,17207.0,17-207,Miscellaneous foods supplement 17-207 and fola...,NaN,(93.9),0.05,...,NaN,NaN,Tr,NaN,NaN,NaN,NaN,NaN,0.00,0.0
3,17-221,Strong ale/barley wine,Mixed sample from different brewers,QA,900.0,50-1103,As MSF 17-221. Additional data from 50-1103,NaN,(86.3),0.11,...,NaN,NaN,Tr,NaN,NaN,NaN,NaN,NaN,0.00,0.0
4,17-220,"Stout, Mackeson",10 samples; canned and bottled,QA,NaN,NaN,NaN,NaN,(92.0),0.06,...,NaN,NaN,Tr,NaN,NaN,NaN,NaN,NaN,NaN,0.0
5,17-219,"Stout, Guinness","10 samples; canned, bottled and draught",QA,17219.0,NaN,Vitamin K1 from Bolton-Smith et al Br.J. Nutr....,NaN,(90.2),0.06,...,NaN,NaN,Tr,NaN,NaN,NaN,NaN,NaN,NaN,0.0
6,17-218,"Shandy, homemade",Recipe. Calculated using equal volumes of lage...,QA,NaN,NaN,NaN,NaN,(94.4),0.03,...,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,0.0
7,17-216,"Pale ale, bottled",Mixed sample from different brewers,QA,897.0,50-1100,As MSF 17-216,NaN,(93.9),0.05,...,NaN,NaN,Tr,NaN,NaN,NaN,NaN,NaN,0.00,0.0
8,17-214,"Lager, premium",10 samples; Carlsberg Special Brew and Heldenb...,QA,17214.0,NaN,NaN,NaN,(88.7),0.05,...,NaN,NaN,Tr,NaN,NaN,NaN,NaN,NaN,NaN,0.0
9,17-213,"Lager, low alcohol","10 samples; Carlton LA, Swan Light, Tennents LA",QA,17213.0,NaN,NaN,NaN,97.0,0.04,...,NaN,NaN,Tr,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [204]:
using = tester[(tester['Group']!='QA')&(tester['Group']!='QI')&(tester['Group']!='QC')&(tester['Group']!='QE')&(tester['Group']!='QF')&(tester['Group']!='QK')&(tester['Group']!='QG')]

In [202]:
using['Water (g)']

/home/kaitlyn/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


48      48      95.9\n49      99.9\n50      81.1\n51  ...
49      48      95.9\n49      99.9\n50      81.1\n51  ...
50      48      95.9\n49      99.9\n50      81.1\n51  ...
51      48      95.9\n49      99.9\n50      81.1\n51  ...
52      48      95.9\n49      99.9\n50      81.1\n51  ...
53      48      95.9\n49      99.9\n50      81.1\n51  ...
54      48      95.9\n49      99.9\n50      81.1\n51  ...
55      48      95.9\n49      99.9\n50      81.1\n51  ...
56      48      95.9\n49      99.9\n50      81.1\n51  ...
57      48      95.9\n49      99.9\n50      81.1\n51  ...
58      48      95.9\n49      99.9\n50      81.1\n51  ...
59      48      95.9\n49      99.9\n50      81.1\n51  ...
60      48      95.9\n49      99.9\n50      81.1\n51  ...
61      48      95.9\n49      99.9\n50      81.1\n51  ...
62      48      95.9\n49      99.9\n50      81.1\n51  ...
63      48      95.9\n49      99.9\n50      81.1\n51  ...
64      48      95.9\n49      99.9\n50      81.1\n51  ...
65      48    